# ***🧠 Dynamic Pricing Engine for E-Commerce***

This project builds an AI-powered pricing engine that recommends the best price for a product based on real-time features like stock, competition, demand, and discount.

Model: **XGBoost Regressor**  
Deployment-ready for use in Android apps or e-commerce dashboards.


## 📊**Create and Export the Sample Pricing Dataset**

We start by creating a **sample dataset** that mimics real-world e-commerce product pricing scenarios.  
This dataset will be used to train our dynamic pricing model.

### Dataset Features:
- `product_id`: Unique identifier for each product
- `category`: Product type (e.g., electronics, clothing, etc.)
- `stock`: Current inventory level
- `original_price`: Listed base price
- `competitor_price`: Price offered by a competitor
- `sales_rating`: Average customer rating (out of 5)
- `last_week_sales`: Units sold in the last week
- `discount_applied`: Discount rate (in decimal)
- `target_price`: The ideal price we want the model to learn to predict

The dataset is converted into a pandas DataFrame and exported as a CSV file (`pricing_data.csv`) for further processing.


In [2]:
import pandas as pd

data = {
    "product_id": [101, 102, 103, 104, 105],
    "category": ["electronics", "clothing", "beauty", "home", "electronics"],
    "stock": [10, 50, 30, 20, 15],
    "original_price": [500, 1200, 300, 700, 450],
    "competitor_price": [480, 1100, 290, 650, 430],
    "sales_rating": [4.2, 3.8, 4.5, 4.0, 4.1],
    "last_week_sales": [10, 35, 25, 20, 15],
    "discount_applied": [0.05, 0.10, 0.03, 0.07, 0.02],
    "target_price": [470, 1080, 285, 660, 425]
}

df = pd.DataFrame(data)
df.to_csv("pricing_data.csv", index=False)

df.head()


,product_id,category,stock,original_price,competitor_price,sales_rating,last_week_sales,discount_applied,target_price
0,101,electronics,10,500,480,4.2,10,0.05,470
1,102,clothing,50,1200,1100,3.8,35,0.10,1080
2,103,beauty,30,300,290,4.5,25,0.03,285
3,104,home,20,700,650,4.0,20,0.07,660
4,105,electronics,15,450,430,4.1,15,0.02,425


**Data Processing + Train-Test Split**

- Convert categorical columns
- Split into training and test sets


In [3]:
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv("pricing_data.csv")

# Convert 'category' to numerical values
df["category"] = df["category"].astype("category").cat.codes

# Input (X) and Output (y)
X = df.drop(["product_id", "target_price"], axis=1)
y = df["target_price"]

# Split for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()


,category,stock,original_price,competitor_price,sales_rating,last_week_sales,discount_applied
4,2,15,450,430,4.1,15,0.02
2,0,30,300,290,4.5,25,0.03
0,2,10,500,480,4.2,10,0.05
3,3,20,700,650,4.0,20,0.07


## 📦 **Install XGBoost Library**

We install the **XGBoost** library, which is a high-performance gradient boosting framework widely used for machine learning tasks, especially for structured/tabular data.

It offers:
- Fast training speed
- High model accuracy
- Built-in support for saving/loading models in `.json` format

We'll use this library to build, train, and deploy our dynamic pricing model.


In [37]:
!pip install xgboost


## 🎯 **Train the XGBoost Model and Evaluate Performance**

We now create and train an **XGBoost Regressor** model on our prepared dataset.

### Model Parameters:
- `n_estimators = 100`: Number of boosting rounds (trees)
- `learning_rate = 0.1`: Step size shrinkage used to prevent overfitting
- `max_depth = 3`: Maximum tree depth to control model complexity
- `random_state = 42`: Seed for reproducibility

### Evaluation:
After training, we evaluate the model on the test dataset using **Mean Squared Error (MSE)** — a common metric to measure prediction accuracy for regression problems.

A **lower MSE** indicates a more accurate model.


In [38]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Create and train the model
model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")


Mean Squared Error: 630378.62


 ## 💾 **Save the Trained Model using XGBoost's `.json` Format**

After training and validating the model, we now save it using **XGBoost’s built-in `save_model()` method**.

### Why `.json`?
- ✅ Safer and more reliable than `.pkl` for deployment
- ✅ Officially supported format by XGBoost
- ✅ Avoids serialization errors during API deployment
- ✅ Compatible with `load_model()` when integrating with FastAPI or mobile apps

The saved model (`dynamic_pricing_model.json`) will be used in our **FastAPI backend** to serve real-time price predictions.


In [43]:
# Use XGBoost's built-in save method
model.save_model("dynamic_pricing_model.json")
print("✅ Model saved using XGBoost format (.json)")


✅ Model saved using XGBoost format (.json)


## 🌐 **Install FastAPI & Uvicorn for API Deployment**

We will now install the necessary libraries to deploy our trained model as a web API using **FastAPI** and **Uvicorn**.

- **FastAPI**: A fast and modern Python web framework for building APIs
- **Uvicorn**: A lightning-fast ASGI server to run the FastAPI app


In [26]:
!pip install fastapi uvicorn


## 🧩 Deploy Trained Model with FastAPI and XGBoost (.json Format)

In this step, we load the trained XGBoost model saved in `.json` format and set up a complete **FastAPI backend**.

### Key Components:
- `XGBRegressor().load_model("dynamic_pricing_model.json")`: Loads the trained model
- `ProductData`: A Pydantic schema to validate incoming request data
- `POST /predict`: Accepts product features and returns the recommended price
- `GET /`: A root route to check if the API is live





In [52]:
from fastapi import FastAPI
from pydantic import BaseModel
import xgboost as xgb
import pandas as pd

# Load the trained model
model = xgb.XGBRegressor()
model.load_model("dynamic_pricing_model.json")

# Define FastAPI app
app = FastAPI()

# Request schema
class ProductData(BaseModel):
    category: int
    stock: int
    original_price: float
    competitor_price: float
    sales_rating: float
    last_week_sales: int
    discount_applied: float

# Home route
@app.get("/")
def read_root():
    return {"message": "Dynamic Pricing Engine is running!"}

# Prediction route
@app.post("/predict")
def predict_price(data: ProductData):
    input_df = pd.DataFrame([data.dict()])
    predicted_price = model.predict(input_df)[0]
    return {"recommended_price": round(predicted_price, 2)}


## 🧪 **Debugging: Check Model File Location**

Now we will verify that the trained `.json` model file is accessible within the current working directory.



In [53]:
import os, glob, pathlib

# Where am I?
print("Current working dir:", os.getcwd())

# Can Colab see the model anywhere?
for p in pathlib.Path(".").rglob("dynamic_pricing_model.json"):
    print("Found model at:", p)


Current working dir: /content
Found model at: dynamic_pricing_model.json


## ✅ **Confirm Model Loaded**

 Now we will print statement to confirm that the trained **XGBoost model** was successfully loaded into memory.

This acts as a checkpoint to ensure the model is ready for prediction and deployment.


In [54]:
print("✅ XGBoost model loaded successfully and ready for prediction.")



✅ XGBoost model loaded successfully and ready for prediction.


## 🧪 **Final Test: Predict Recommended Price Using Sample Input**

In this final step, we simulate a real-world pricing scenario using a single test input (as it would be passed from an app or frontend).

We prepare the input as a dictionary, convert it into a DataFrame, and pass it to the trained XGBoost model to get the predicted recommended price.

### Sample Input:
- `category`: 0 (Encoded category ID)
- `stock`: 10
- `original_price`: ₹500
- `competitor_price`: ₹480
- `sales_rating`: 4.2
- `last_week_sales`: 10
- `discount_applied`: 5%

This test verifies that the model is working properly and ready to be integrated into an API or app.


In [55]:
import pandas as pd

input_data = {
    "category": 0,
    "stock": 10,
    "original_price": 500,
    "competitor_price": 480,
    "sales_rating": 4.2,
    "last_week_sales": 10,
    "discount_applied": 0.05
}

df = pd.DataFrame([input_data])
price = model.predict(df)[0]
print("Predicted Recommended Price:", round(price, 2))


Predicted Recommended Price: 286.04


---

## ✅ **Project Complete: Dynamic Pricing Engine**

This notebook successfully demonstrates the complete development of an AI-powered **Dynamic Pricing Engine** using:

- 🧠 XGBoost for model training
- 📊 Custom structured dataset
- 🚀 FastAPI backend (for deployment)
- 📱 Android/web-ready JSON format model

The model is now fully trained, tested, and saved in deployment format.  


---

### 📌 Created by: **Mayank Pratap Singh**(Self-Taught AI/ML Developer | Project-Based Learner | Building Real Solutions)


